In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

def calculate_shortest_stage_order(stage_df):
    G = nx.Graph()

    for _, row in stage_df.iterrows():
        G.add_node(row['stage'], latitude=row['stage_latitude'], longitude=row['stage_longitude'])

    for node1, data1 in G.nodes(data=True):
        for node2, data2 in G.nodes(data=True):
            if node1 != node2:
                lat1, lon1 = data1['latitude'], data1['longitude']
                lat2, lon2 = data2['latitude'], data2['longitude']
                distance = ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5
                G.add_edge(node1, node2, weight=distance)

    shortest_path = nx.approximation.traveling_salesman_problem(G, cycle=True)

    shortest_df = pd.DataFrame(shortest_path, columns=['stage'])
    shortest_df['rank'] = shortest_df.index + 1
    return shortest_df

# Read the files
date_parser = lambda x: pd.to_datetime(x, format='%m/%d/%y %I.%M %p', errors='coerce')
schedule_df = pd.read_csv('schedule.csv', parse_dates=['start_date', 'end_date'], date_parser=date_parser)
favorite_df = pd.read_csv('favorite.csv')

# 2. Join the data based on artist_name
merged_df = schedule_df.merge(favorite_df, on='artist_name')

merged_df['round_start_date'] = merged_df['start_date'].dt.floor('H')
merged_df['round_end_date'] = merged_df['end_date'].dt.ceil('H')

min_start_dates = merged_df.groupby('round_start_date')['start_date'].min().reset_index()
max_end_dates = merged_df.groupby('round_end_date')['end_date'].max().reset_index()

# Merge the minimum start_date and maximum end_date back into the original DataFrame
merged_df = merged_df.merge(min_start_dates, on='round_start_date', suffixes=('', '_min'))
merged_df = merged_df.merge(max_end_dates, on='round_end_date', suffixes=('', '_max'))

# Sort the DataFrame by the 'Datetime' column in ascending order
merged_df.sort_values(by='round_start_date', inplace=True)

# Add a new column for ranking using the 'rank' method with 'method='min''
merged_df['time_order'] = merged_df['round_start_date'].rank(method='min').astype(int)
merged_df

,artist_name,start_date,end_date,stage,stage_latitude,stage_longitude,day,round_start_date,round_end_date,start_date_min,end_date_max,time_order
1,Rizky Febian,2023-09-25 17:00:00,2023-09-25 17:45:00,Gegap Gempita,-6.219201,106.800470,1,2023-09-25 17:00:00,2023-09-25 18:00:00,2023-09-25 17:00:00,2023-09-25 17:45:00,1
2,Endah n Rhesa,2023-09-25 18:00:00,2023-09-25 18:35:00,Pop Up Stage,-6.217073,106.807393,1,2023-09-25 18:00:00,2023-09-25 19:00:00,2023-09-25 18:00:00,2023-09-25 18:35:00,2
3,Padi Reborn,2023-09-25 19:30:00,2023-09-25 20:30:00,Pestapora IM3,-6.217197,106.799488,1,2023-09-25 19:00:00,2023-09-25 21:00:00,2023-09-25 19:30:00,2023-09-25 20:40:00,3
4,Danilla,2023-09-25 19:30:00,2023-09-25 20:15:00,Riang Gembira,-6.217557,106.801023,1,2023-09-25 19:00:00,2023-09-25 21:00:00,2023-09-25 19:30:00,2023-09-25 20:40:00,3
5,Pamungkas,2023-09-25 19:55:00,2023-09-25 20:40:00,Gegap Gempita,-6.219201,106.800470,1,2023-09-25 19:00:00,2023-09-25 21:00:00,2023-09-25 19:30:00,2023-09-25 20:40:00,3
...,...,...,...,...,...,...,...,...,...,...,...,...
30,HIVI!,2023-09-28 17:00:00,2023-09-28 17:45:00,Pestapora IM3,-6.217197,106.799488,2,2023-09-28 17:00:00,2023-09-28 18:00:00,2023-09-28 17:00:00,2023-09-28 17:45:00,31
31,Sore,2023-09-28 18:00:00,2023-09-28 18:45:00,Riang Gembira,-6.217557,106.801023,2,2023-09-28 18:00:00,2023-09-28 19:00:00,2023-09-28 18:00:00,2023-09-28 18:45:00,32
33,Nadin Amizah,2023-09-28 20:45:00,2023-09-28 21:30:00,Gegap Gempita,-6.219201,106.800470,2,2023-09-28 20:00:00,2023-09-28 22:00:00,2023-09-28 20:00:00,2023-09-28 21:30:00,33
32,Ada Band,2023-09-28 20:00:00,2023-09-28 20:45:00,Hingar Bingar,-6.219257,106.799537,2,2023-09-28 20:00:00,2023-09-28 21:00:00,2023-09-28 20:00:00,2023-09-28 20:45:00,33


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Define the desired duration in minutes
desired_duration_minutes = 20

# Function to calculate optimal duration for a single row
def calculate_optimal_duration(row):
    start_date = datetime.strptime(row["start_date"], "%d/%m/%y %H.%M")
    end_date = datetime.strptime(row["end_date"], "%d/%m/%y %H.%M")

    # Calculate the number of intervals needed
    num_intervals = int((end_date - start_date).total_seconds() / (desired_duration_minutes * 60))

    # Calculate the actual duration in seconds
    actual_duration_seconds = (end_date - start_date).total_seconds()

    # Calculate the optimal duration in seconds
    optimal_duration_seconds = actual_duration_seconds / num_intervals

    # Convert the optimal duration to minutes and return it
    optimal_duration_minutes = optimal_duration_seconds / 60
    return f"{optimal_duration_minutes:.2f}"

# Apply the function to each row and create a new column "intervals"
#merged_df["intervals"] = .apply(calculate_optimal_duration, axis=1)
# Filter unique duplicated records based on the "time_order" column
unique_duplicated_records = merged_df[merged_df["time_order"].duplicated(keep=False)].drop_duplicates()
unique_duplicated_records

,artist_name,start_date,end_date,stage,stage_latitude,stage_longitude,day,round_start_date,round_end_date,start_date_min,end_date_max,time_order
3,Padi Reborn,2023-09-25 19:30:00,2023-09-25 20:30:00,Pestapora IM3,-6.217197,106.799488,1,2023-09-25 19:00:00,2023-09-25 21:00:00,2023-09-25 19:30:00,2023-09-25 20:40:00,3
4,Danilla,2023-09-25 19:30:00,2023-09-25 20:15:00,Riang Gembira,-6.217557,106.801023,1,2023-09-25 19:00:00,2023-09-25 21:00:00,2023-09-25 19:30:00,2023-09-25 20:40:00,3
5,Pamungkas,2023-09-25 19:55:00,2023-09-25 20:40:00,Gegap Gempita,-6.219201,106.800470,1,2023-09-25 19:00:00,2023-09-25 21:00:00,2023-09-25 19:30:00,2023-09-25 20:40:00,3
8,Yura Yunita,2023-09-25 21:10:00,2023-09-25 21:55:00,Gegap Gempita,-6.219201,106.800470,1,2023-09-25 21:00:00,2023-09-25 22:00:00,2023-09-25 21:00:00,2023-09-25 22:00:00,7
7,The Adams,2023-09-25 21:00:00,2023-09-25 21:45:00,Pestapora IM3,-6.217197,106.799488,1,2023-09-25 21:00:00,2023-09-25 22:00:00,2023-09-25 21:00:00,2023-09-25 22:00:00,7
...,...,...,...,...,...,...,...,...,...,...,...,...
25,Aldi Taher & Band,2023-09-27 23:15:00,2023-09-28 00:15:00,Riang Gembira,-6.217557,106.801023,3,2023-09-27 23:00:00,2023-09-28 01:00:00,2023-09-27 23:15:00,2023-09-28 00:55:00,25
26,The Changcuters,2023-09-27 23:50:00,2023-09-28 00:35:00,Boss Stage,-6.217308,106.804468,3,2023-09-27 23:00:00,2023-09-28 01:00:00,2023-09-27 23:15:00,2023-09-28 00:55:00,25
27,Dewa 19 feat Ello,2023-09-27 23:55:00,2023-09-28 00:55:00,Pestapora IM3,-6.217197,106.799488,3,2023-09-27 23:00:00,2023-09-28 01:00:00,2023-09-27 23:15:00,2023-09-28 00:55:00,25
33,Nadin Amizah,2023-09-28 20:45:00,2023-09-28 21:30:00,Gegap Gempita,-6.219201,106.800470,2,2023-09-28 20:00:00,2023-09-28 22:00:00,2023-09-28 20:00:00,2023-09-28 21:30:00,33
